# WEB SCRAPING

In [1]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from datetime import date
import re

In [2]:
# Get your own User-Agent from https://httpbin.org/get
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

today = date.today() 

In [3]:
baseurl = 'https://ie.trustpilot.com'

productlinks = []
for x in range(1,4):
    r = requests.get(f'https://ie.trustpilot.com/review/www.murphylarkin.com?page={x}')
    soup = BeautifulSoup(r.content, 'html.parser')
    productlist = soup.find_all('div', class_='styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ')

    for item in productlist:
        link = item.find('a', href=True)
        if link:
            productlinks.append(baseurl + link['href'])

print(productlinks)

['https://ie.trustpilot.com/users/6022493bb3ed6200192de8d7', 'https://ie.trustpilot.com/users/5a4eb5390000ff000afb0bfd', 'https://ie.trustpilot.com/users/64593d30b352d30013e8a8ae', 'https://ie.trustpilot.com/users/5b77f1604de5666d34a1416c', 'https://ie.trustpilot.com/users/5d77c3a876bf9c22e82d5c0a', 'https://ie.trustpilot.com/users/607f06bf92d1ea001a18799e', 'https://ie.trustpilot.com/users/5ead583746f4d0297ff8bab6', 'https://ie.trustpilot.com/users/606c8dc53aeffa001b407254', 'https://ie.trustpilot.com/users/645a70db3d0add0012854282', 'https://ie.trustpilot.com/users/620bb3190dd4fc0012c240c3', 'https://ie.trustpilot.com/users/5f9d3956f0f73b001ad1d367', 'https://ie.trustpilot.com/users/6405e99be15124001414ab5b', 'https://ie.trustpilot.com/users/64266ca3f46d15001358d58f', 'https://ie.trustpilot.com/users/607572967cb93e001a973947', 'https://ie.trustpilot.com/users/5e999e1cd7ff731bd462f22d', 'https://ie.trustpilot.com/users/5434edf400006400018ee5ca', 'https://ie.trustpilot.com/users/59346b

In [4]:
Reviews = []
for link in productlinks:
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'lxml')
    
    # finding line where is the 'name' of product, removing unwanted characters behind name
    na = soup.find("span", class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17")
    # Extract the name
    name = na.get_text(strip=True)
    
    no = soup.find("span", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l")
    Number = no.get_text(strip=True)
    Number2 = Number.replace("reviews", "").replace("review", "")

    if soup.find(text='Verified'):
        verified = 'Yes'
    else:
        verified = 'No'
        
    t = soup.find('h2', class_='typography_heading-s__f7029 typography_appearance-default__AAY17')
    title = t.get_text(strip=True)
    
    re = soup.find('p', class_ = 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn')
    Review = re.get_text(strip=True)
    

    # creating a dictionary to save data
    rev = {
        'Date': today,
        'Name' : name,
        'Verified':verified,
        'No of Reviews':Number2,
        'Title': title,
        'Review':Review}
    Reviews.append(rev)
    print(rev)

{'Date': datetime.date(2023, 5, 9), 'Name': 'Emmet Gaule', 'Verified': 'Yes', 'No of Reviews': '2', 'Title': 'Great service in the Tramore branch.', 'Review': 'Order placed online. Follow up call with me to confirm everything. Ready for collection on date stated. Very professional service.'}
{'Date': datetime.date(2023, 5, 9), 'Name': 'Gavin Tuite', 'Verified': 'Yes', 'No of Reviews': '3', 'Title': 'Good service and delivery however quite…', 'Review': 'Good service and delivery however quite a few defects in the primed timber , chips and gouges and paint overspray . Architrave needed a good sand before installation. Replacement boards were provided free of charge when I provided photos of the damaged boards so I would buy there again'}
{'Date': datetime.date(2023, 5, 9), 'Name': "Stephen O'Neill", 'Verified': 'Yes', 'No of Reviews': '1', 'Title': 'Very prompt delivery', 'Review': 'Very prompt delivery. Even followed up to make sure parts were delivered and everything was in order'}
{'D

In [5]:
# transforming dictionary into DF
df = pd.DataFrame(Reviews)
print(df.head(10))

         Date              Name Verified No of Reviews  \
0  2023-05-09       Emmet Gaule      Yes             2   
1  2023-05-09       Gavin Tuite      Yes             3   
2  2023-05-09   Stephen O'Neill      Yes             1   
3  2023-05-09             Gerry      Yes             9   
4  2023-05-09              Lucy      Yes            10   
5  2023-05-09                AZ      Yes             2   
6  2023-05-09         Margarita      Yes             2   
7  2023-05-09  Michael Kavanagh      Yes             2   
8  2023-05-09        Peter Cork      Yes             1   
9  2023-05-09            Mike H      Yes             3   

                                              Title  \
0              Great service in the Tramore branch.   
1          Good service and delivery however quite…   
2                              Very prompt delivery   
3               Product arrived quickly and I even…   
4                        great experience shopping…   
5                   Second purc

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           60 non-null     object
 1   Name           60 non-null     object
 2   Verified       60 non-null     object
 3   No of Reviews  60 non-null     object
 4   Title          60 non-null     object
 5   Review         60 non-null     object
dtypes: object(6)
memory usage: 2.9+ KB


In [7]:
df.head()

,Date,Name,Verified,No of Reviews,Title,Review
0,2023-05-09,Emmet Gaule,Yes,2,Great service in the Tramore branch.,Order placed online. Follow up call with me to...
1,2023-05-09,Gavin Tuite,Yes,3,Good service and delivery however quite…,Good service and delivery however quite a few ...
2,2023-05-09,Stephen O'Neill,Yes,1,Very prompt delivery,Very prompt delivery. Even followed up to make...
3,2023-05-09,Gerry,Yes,9,Product arrived quickly and I even…,Product arrived quickly and I even received a ...
4,2023-05-09,Lucy,Yes,10,great experience shopping…,As always a great experience shopping with The...
